# Pipeline to run all files
* ~~All scripts currently output Excel files because we were working independently~~
* ~~<b>We'll be changing this so that each script passes dataframes until the end of the pipeline.</b>~~
* Our plan is to not store anything (refresh daily) but need to consider how to deal with taking in user feedback
* Porter

In [26]:
#Gather Articles
import NewsAPI as news
import os

#delete any old sorted article files
#if os.path.exists("articles.csv"):
#    os.remove("articles.csv")

#inputs in order: set manual date (yes=1/no=0, no = last 24 hours of news), manual start date as "YYYY-MM-DD"(leave empty if 0), manual end date (leave empty if 0, companies to pull (6= all 19 companies)
articleDB = news.main(1,"2019-02-25","2019-03-01",6) #output is called 'NewsAPIOutput.xlsx' in Python Scripts > Data folder

Gathering articles on (Gap Inc) OR (Foot Locker) OR (L Brands) OR Macerich OR Kimco OR TJX OR CVS OR (Home Depot) OR (Best Buy) OR (Lowe's) OR Walmart OR (Target's) OR TGT OR Amazon OR Kroger OR Walgreens OR Kohl's OR (Dollar General) OR (Bed Bath and Beyond) OR Safeway from: 2019-02-25 to 2019-03-01
997


In [27]:
#Article Cleaning (must pip install tqdm first (only once) to run)
import dataClean as dc
articleDB = dc.DataClean(articleDB)

In [28]:
#Feature Selection and Binary Article Encoding
import FeatureEncoding as fe
contentBinaryMatrix = fe.encoding(0, df=articleDB, text_col='content', norm='prStem')
titleBinaryMatrix = fe.encoding(0, df=articleDB, text_col = 'title', norm='prStem')

content
prStem
Binary Encoding
False
title
prStem
Binary Encoding
False


In [29]:
#Logistic Regression Classifier + Article Ranking, complete final file is called 'results_encoding.xlsx'
import logReg as lr
articleDB = lr.runLogReg(titleBinaryMatrix, contentBinaryMatrix, articleDB)
#articleDB = runLogReg(titleBinaryMatrix, contentBinaryMatrix, articleDB)

Index([0, 1, 'article_id', 'prediction'], dtype='object')
                                               title  \
0  Amazon could be gearing up to reveal Walmart, ...   
1  Amazon is planning to open new grocery chain, ...   
2  CNBC'S CHRISTINA FARR & EUGENE KIM: AMAZON TAP...   
3    Week ahead: US jobs, retail earnings, ECB rates   
4         And America's least favorite stores are...   

                                         description  
0  Amazon is reportedly planning to launch a new ...  
1  Amazon, which shook the grocery industry when ...  
2                                                     
3  While US-China trade talks continue to hold ce...  
4  When it comes to satisfying their customers, m...  


In [30]:
# This code extracts and ranks "tags" + counts frequency of tag mentions in articles 
"""
Inputs: 
    1) articleDB - uses column 'content'
    2) (optional) - tag type (could be 'ngrams'{unlimited}, 'bigrams'{terms with up to 2 words}, or 'unigrams'{single terms})
        - default is 'bigrams'
        - future work: add in noun phrases, named entities 
Outputs:
    1) articleDB = articleDB with appended columns "tags" and "tags_top_5"
    2) trendingTermsDB = keyterms by # article mentions
"""
import ContextExtraction as ce
articleDB, trendingTermsDB = ce.retrieveContext(articleDB)

100%|████████████████████████████████████████████████████████████████████████████████| 578/578 [03:14<00:00,  1.77it/s]


In [31]:
import EncodingforRecommender as rec
articleDB = rec.recommender(articleDB)

tifidf Encoding
bin Encoding


100%|███████████████████████████████████████████████████████████████████████████████| 578/578 [00:01<00:00, 418.87it/s]


tf Encoding


100%|███████████████████████████████████████████████████████████████████████████████| 578/578 [00:01<00:00, 426.83it/s]


       amazon  purchase     whole     foods      sent     shock  waves  \
0    0.070707   0.02439  0.096154  0.227273  0.021277  0.090909    0.2   
1    0.060606   0.02439  0.096154  0.363636  0.000000  0.000000    0.0   
2    0.090909   0.04878  0.019231  0.045455  0.000000  0.000000    0.0   
3    0.000000   0.00000  0.000000  0.000000  0.000000  0.000000    0.0   
4    0.015152   0.00000  0.019231  0.045455  0.000000  0.000000    0.0   
5    0.000000   0.02439  0.000000  0.000000  0.000000  0.000000    0.0   
6    0.010101   0.00000  0.000000  0.000000  0.000000  0.000000    0.0   
7    0.000000   0.00000  0.000000  0.000000  0.000000  0.000000    0.0   
8    0.075758   0.02439  0.000000  0.000000  0.000000  0.000000    0.0   
9    0.015152   0.00000  0.038462  0.045455  0.000000  0.000000    0.0   
10   0.000000   0.00000  0.000000  0.000000  0.000000  0.000000    0.0   
11   0.000000   0.00000  0.000000  0.000000  0.000000  0.000000    0.0   
12   0.005051   0.00000  0.000000  0.0

[578 rows x 22197 columns]


In [32]:
import frontPage as fp
frontpage = fp.FrontPage(articleDB, trendingTermsDB)

In [33]:
import pandas as pd
articles = pd.DataFrame(frontpage['articles'])
topterms= pd.DataFrame(frontpage['topterms'])
articles.columns

Index(['date', 'origContent', 'related_articles', 'source', 'tags', 'title'], dtype='object')

In [34]:
titles = pd.DataFrame(articles['tags'])

In [35]:
pd.DataFrame.to_excel(articleDB, 'Main Version-25-01.xlsx')